# Implement a function to query json_schema

The goal of this excercise is to implement a function that allows you to query the type of a keypath in a jsonschema. This function will accept a valid jsonschema (https://json-schema.org/specification.html) as dict, a key_path (eg: `foo.bar.baz`) and return the `type` of the property. 

Note:

1. There are only two fields in the schema you have to pay attention to: `properties` and `definitions`.
2. if the dictionary associated with the field has a field named `$ref` it means that its refering to another schema stored under the top level schema. You have follow the link to get to the actual definition. 
3. For the sake of this excercise you can assume that all values for `$ref` will start wiht `#/<key_path>`.
4. You should see a schema and some assert statements under "Test" section. You should NOT have to change anything under the "Test" section. If you can get our code to pass the tests, then it means your function works as expected.

Feel free to use whatever libraries you need to use except any libraries that actually allow you to query the jsonschema

In [42]:
def get_ref(schema, key_path):
    """ Find the dict with key_path value.
    This is a called function by get_property.
    
    Keyword Arguments:
    schema -- the json string converted to a python object
    key_path -- the key.key.key string format containing the property type
    """
    node = schema
    for segment in key_path.split('/')[1:]:
        node = node[segment]
    return node
    
def get_property(schema, key_path):
    """ Find the dict with key_path value in key.
    This function calls get_ref when key_path is nested.
    This function is called by get_type_for_key_path
    """
    node = schema
    for segment in key_path.split('.'):
        if '$ref' in node:
            node = get_ref(schema, node['$ref'])
        node = node['properties'][segment]
    return node


def get_type_for_key_path(schema, key_path):
    """ Returns the 'type' key of the key_path"""
    try:
        return get_property(schema, key_path)['type']
    except KeyError:
        return None
    

# Test

In [43]:
import json

schema = json.loads('''{
  "$id": "https://example.com/nested-schema.json",
  "title": "nested-schema",
  "$schema": "http://json-schema.org/draft-07/schema#",
  "required": [
    "EmploymentInformation",
    "EmployeePartyID",
    "Age"
  ],
  "properties": {
    "EmployeePartyID": {
      "type": "string",
      "minLength": 1,
      "maxLength": 3
    },
    "EmploymentInformation": {
      "$ref": "#/definitions/EmploymentInformation"
    },
    "Age": {
      "type": "integer",
      "minimum": 16,
      "maximum": 80
    }
  },
  "definitions": {
    "EmploymentInformation": {
      "type": "object",
      "required": [
        "OriginalHireDate"
      ],
      "properties": {
        "OriginalHireDate": {
          "type": "string",
          "format": "date"
        },
        "Beneficiary": {
          "$ref": "#/definitions/DependantInformation"
        }
      }
    },
    "DependantInformation": {
      "type": "object",
      "required": [
        "Name"
      ],
      "properties": {
        "Name": {
          "type": "string",
          "minLength": 5
        }
      }
    }
  },
  "description": "nested-schema"
}''')


assert(get_type_for_key_path(schema, "Age") == "integer")
assert(get_type_for_key_path(schema, "EmploymentInformation.OriginalHireDate") == "string")
assert(get_type_for_key_path(schema, "EmploymentInformation.Beneficiary.Name") == "string")
assert(get_type_for_key_path(schema, "foo.bar") == None)